# CSV Crawler-Updater

This script will go into a folder of your choice and update the data in all the csv files in that folder. Mainly it will check for certain conditions in rows of certain columns and delete rows. It also reads a spreadsheed that it uses to update a column. The reference spreadsheet contains the old codes corresponding to new codes. It will swap the old codes with the new codes.

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import re
import glob
import csv

### Input Variables

In [10]:
# Directory of the csv files you want to process
Input_path_CSVs = 'D:/FILES/input_csvs/'

# Can change to xlsx if needed, other changes will be nessesary to code
extension = 'csv'

# Csv files seperator for input and output files...    , or |
delimiter = '|'

# Output of the CSV files
Output_path_CSVs = 'D:/FILES/output_csv_processed/'

# Output path of bad CSV files
Output_path_badCSVs = 'D:/FILES/output_csv_bad/'

### Save CSV Filenames in a List and Dataframe

In [11]:
# Get the csv filenames into an array
os.chdir(Input_path_CSVs)
filenames = [i for i in glob.glob('*.{}'.format(extension))]

# Get the number of csv files
NumFiles = len(filenames)
print(NumFiles, 'files will be processed')

1006 files will be processed


### Find which files dont have 'LOCATIONCODE' or 'LocationDescription' columns

In [7]:
counter_good_files = 0
counter_bad_files = 0
List_Unsupported_Files = []
for filename in filenames:
    # Save an individual file as a DataFrame Object to analyse
    try:
        df_file = pd.read_csv(filename, sep=delimiter, index_col=False, engine='python')
        if ('LOCATIONCODE' in df_file.columns) and ('LocationDescription' in df_file.columns):
            counter_good_files +=1
        else:
            List_Unsupported_Files.append(filename)
            counter_bad_files +=1
    except:
        List_Unsupported_Files.append(filename)
        counter_bad_files +=1
        
print('counter_good_files:')
print(counter_good_files)
print('counter_bad_files:')
print(counter_bad_files)
print('Unsupported Files:')
print(List_Unsupported_Files[0])
print(List_Unsupported_Files[1])
print(List_Unsupported_Files[2])
print(List_Unsupported_Files[3])

counter_good_files:
1006
counter_bad_files:
364
Unsupported Files:
ALS_Drinking_Water_Historic_1030_15-47164_23773.csv
ALS_Drinking_Water_Historic_1098_14-16240_61047.csv
ALS_Drinking_Water_Historic_1099_14-16342_61049.csv
ALS_Drinking_Water_Historic_1244_13-02125_61375.csv


### Move Files with Weird Column Structure to Weird Folder (will need manual processing)

In [8]:
import shutil
import os

source = Input_path_CSVs
move_folder = 'D:/FILES/output_csv_bad_column_structure/'

files = os.listdir(source)

for f in files:
    if f in List_Unsupported_Files:
        shutil.move(f, move_folder)

### Loop though each file and Process it, Create Excel Report on Changes Made

In [12]:
# Get the csv filenames into an array
os.chdir(Input_path_CSVs)
filenames = [i for i in glob.glob('*.{}'.format(extension))]

#Create an Empty Dataframe for Report Numbers
List_Columns = ['Filename', 'Number of Duplicate Rows', 'Number of Retests', 'Number of QA Data Rows', 'Number of Bad SPTs']
df_Report = pd.DataFrame(columns=List_Columns)
df_Report.head()


### START LOOP #### Loop through each csv file in the input directory
for filename in filenames:
    # Set File Identifiers to False Untill proven True
    QA_Data_In_File = False
    Bad_sptz = False
    Dups_In_File = False
    Exact_Dups_In_File = False
    
    Int_Retest_Rows = int(0)
    Int_QA_Rows = int(0)
    Int_Bad_SPTz = int(0)
    Int_Dup_Rows = int(0)

    # Save an individual file as a DataFrame Object to analyse
    df_file = pd.read_csv(filename, sep=delimiter, index_col=False, engine='python')
    # Delete Rows with everything missing
    df_file = df_file.dropna(axis='index', how='all')

    # Check and Find if Blanks exist in Location Code Rows
    bools_ml = df_file['LOCATIONCODE'].isnull()
    bools_ml = np.array(bools_ml)

    # Check and Find if Blanks exist in Location Description Rows
    bools_md = df_file['LocationDescription'].isnull()
    bools_md = np.array(bools_ml)

    # Find Quality Assurance Data Rows
    Series_Loc_Desc = df_file['LocationDescription']
    Series_Loc_Desc = Series_Loc_Desc.fillna(' ')
    bools_bd = Series_Loc_Desc == 'Blind Dup A'
    bools_bd = np.array(bools_bd)
    bools_bd = np.logical_and(bools_ml, bools_bd)
    bools_fb = Series_Loc_Desc == 'Field Blank'
    bools_fb = np.array(bools_fb)
    bools_fb = np.logical_and(bools_ml, bools_fb)
    bools_qa = np.logical_or(bools_bd, bools_fb)    
    bools_filter_QA = np.invert(bools_qa)
    if False in bools_filter_QA:
        QA_Data_In_File = True

    # Fixing QA Data and update DataFrame object if neccesary
    if QA_Data_In_File == True:
        # Number of QA Rows in data
        Int_QA_Rows = np.sum(bools_qa)
        # Filter Out Quality Control Data and Save the result as a New DataFrame Object
        df_file = df_file[bools_filter_QA]
        Files_QA_Data.append(filename)

    # Check if SPT's still don't exist in Location Code Rows
    Array_Loc_Codes = bools_good_sptz = df_file['LOCATIONCODE']
    Array_Loc_Codes = Array_Loc_Codes.fillna(' ')
    bools_good_sptz = Array_Loc_Codes.str.startswith('SPT')
    bools_good_sptz = np.array(bools_good_sptz)

    
    # Save bad SPT codes in data to a new dataframe
    if False in bools_good_sptz:
        Bad_sptz = True
        # Create dataframe for stuff that can't load
        df_cant_load = df_file
        bools_filter_badSPT = np.invert(bools_good_sptz)
        # Count the number of Bad SPTz
        Int_Bad_SPTz = np.sum(bools_filter_badSPT)
        # Leave only Bad SPT data in New Dataframe
        df_cant_load = df_cant_load[bools_filter_badSPT]        
        # Filter Orginal Dataframe to banish SPTs that won't load
        df_file = df_file[bools_good_sptz]
    
    # Create Lists to Check for duplicates
    list_checker_results = []
    for index, row in df_file.iterrows():
        ##string_checker =  str(row['LOCATIONCODE']) + str(row['SAMPLEDATE']) + str(row['TEST_KEY_CODE'])
        string_checker_results =  str(row['LOCATIONCODE']) + str(row['SAMPLEDATE']) + str(row['TEST_KEY_CODE']) + str(row['RESULT'])
        # Only add rows that are not blanks to the list of rows to check for exact duplicates
        if not(string_checker_results.startswith('nan')):
            list_checker_results.append(string_checker_results)

    # Create Array's for Exact duplicates
    Array_Raw_Items_Results = np.array(list_checker_results)
    
    # Decide if there are any exact duplicates    
    if len(np.unique(Array_Raw_Items_Results)) != len(Array_Raw_Items_Results):
        Exact_Dups_In_File = True
    
    # Find the rows where the exact duplicates live
    if Exact_Dups_In_File == True:
        # Create a dictionary of duplicate checker keys vs numvber of duplicates
        unique, counts = np.unique(Array_Raw_Items_Results, return_counts=True)
        Dict_Unique_Counts = dict(zip(unique, counts))
        #Create a list of keys for duplicates only
        List_Duplicate_Keys = ([key for key, val in Dict_Unique_Counts.items() if val > 1])
        # Find which Row are exact duplicates and add to list to delete
        List_Indexes_To_Delete = []
        for key in List_Duplicate_Keys:
            List_Indexes = [i for i, j in enumerate(list_checker_results) if j == key]
            # Remove Fist Duplicate from List
            List_Indexes.pop(0)
            # Append the rows to delete to the List
            List_Indexes_To_Delete.extend(List_Indexes)
        print('List Indexes to Delete')
        # Count the number of Duplicates that will be deleted for Report
        Int_Dup_Rows = len(List_Indexes_To_Delete)
        # Delete the rows that are duplicates
        print(List_Indexes_To_Delete)
        df_file = df_file.drop(df_file.index[List_Indexes_To_Delete])
    
    # Append Update the Report Dataframe
    List_Row_Report = [filename, Int_Dup_Rows, Int_Retest_Rows, Int_QA_Rows, Int_Bad_SPTz]
    df_Temp_Report = pd.DataFrame([List_Row_Report], columns=List_Columns)
    df_Report = df_Report.append(df_Temp_Report, ignore_index=True)
  
    # Create New Processed File if 1 or more rows
    if df_file.shape[0] > 0:
        new_filename = filename[:-4] + '-processed' + filename[-4:]
        Output_filename = Output_path_CSVs + new_filename
        df_file.to_csv(path_or_buf=Output_filename, sep='|', index=False)
        
    # Create New Bad File for fixes that can't be made
    if (Bad_sptz == True):
        if df_cant_load.shape[0] > 0:
            new_filename = filename[:-4] + '-SPT-issue' + filename[-4:]
            Output_filename = Output_path_badCSVs + new_filename
            df_cant_load.to_csv(path_or_buf=Output_filename, sep='|', index=False)   


                  
######  END  LOOP #####

# Generate Excel Report
writer = pd.ExcelWriter('Report.xlsx')
df_Report.to_excel(writer,'Sheet1')
writer.save()


df_Report

List Indexes to Delete
[65, 41, 35, 51, 37, 55, 67, 59, 63, 53, 45, 57, 47, 43, 39, 61, 49]
List Indexes to Delete
[26, 28]
List Indexes to Delete
[60]
List Indexes to Delete
[33, 31, 157, 43, 32, 30, 44, 37, 39, 34, 40, 35, 160, 155, 158, 41, 154, 45]
List Indexes to Delete
[59]
List Indexes to Delete
[3]
List Indexes to Delete
[35, 34]
List Indexes to Delete
[18]
List Indexes to Delete
[15]
List Indexes to Delete
[7]
List Indexes to Delete
[7]
List Indexes to Delete
[7]
List Indexes to Delete
[13]
List Indexes to Delete
[9]
List Indexes to Delete
[13]
List Indexes to Delete
[16]
List Indexes to Delete
[15]
List Indexes to Delete
[11]
List Indexes to Delete
[12]
List Indexes to Delete
[13]
List Indexes to Delete
[15]
List Indexes to Delete
[6]
List Indexes to Delete
[27]
List Indexes to Delete
[65, 62, 68, 69, 96, 58, 59, 63, 104, 54, 70, 56, 67, 103, 111, 100, 106, 72, 57, 66, 105, 64, 99, 110, 107, 97, 109, 71, 121, 108, 73, 74, 98, 101, 61, 60, 102, 112, 55]


,Filename,Number of Duplicate Rows,Number of Retests,Number of QA Data Rows,Number of Bad SPTs
0,ALS_DRINKING-ALGAE_WATER_201703101541_17-07346...,0.0,0.0,0.0,0.0
1,ALS_DRINKING-ALGAE_WATER_201703101541_17-07346...,0.0,0.0,0.0,0.0
2,ALS_DRINKING-ALGAE_WATER_201703101541_17-07346...,0.0,0.0,0.0,0.0
3,ALS_DRINKING-ALGAE_WATER_201703101541_17-07371...,0.0,0.0,0.0,0.0
4,ALS_DRINKING-ALGAE_WATER_201703101541_17-07371...,0.0,0.0,0.0,0.0
5,ALS_DRINKING-ALGAE_WATER_201703101541_17-11320...,0.0,0.0,0.0,0.0
6,ALS_DRINKING-ALGAE_WATER_201703151012b_95813.csv,17.0,0.0,0.0,0.0
7,ALS_DRINKING-ALGAE_WATER_201704131557_DL90070_...,0.0,0.0,0.0,0.0
8,ALS_DRINKINGWATER_ALGAE11-07993_88400.csv,0.0,0.0,0.0,0.0
9,ALS_DRINKINGWATER_ALGAE11-08628_88410.csv,0.0,0.0,0.0,0.0
